# Hardware Emit Pass
The `emit_verilog` transform pass generates a top-level RTL file and testbench file according to the `MaseGraph`, which includes a hardware implementation of each layer in the network. This top-level file instantiates modules from the `components` library in MASE and/or modules generated using [HLS](https://en.wikipedia.org/wiki/High-level_synthesis), when internal components are not available. The hardware can then be simulated using [Verilator](https://www.veripool.org/verilator/), or deployed on an FPGA.

First, add Machop to your system PATH (if you haven't already done so) and import the required libraries.

In [1]:
import os, sys
import torch
torch.manual_seed(0)

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_cocotb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn

# TO DO: remove
import os
os.environ["PATH"] = "/opt/homebrew/bin:" + os.environ["PATH"]
!verilator

/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to debug


Usage:
        verilator --help
        verilator --version
        verilator --binary -j 0 [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --cc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --sc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --lint-only -Wall [source_files.v]...



# CNN Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self, num_classes=10):
    # initialize the parent class—in this case, the nn.Module class from PyTorch.
    super().__init__()

    # Define layers in the model
    # When you instantiate a layer like nn.Conv2d inside the forward method,
    # it is created anew each time forward is called. Because that layer is constructed at runtime during every forward pass,
    # its weights are randomly re-initialized every time.
    # Hence the weights will not be tracked by PyTorch’s parameter-registration mechanism (they won’t show up in model.parameters()), and
    # Won’t get updated during backpropagation (since they’re re-created each time),
    # Are randomly re-initialized on every forward pass, which makes learning impossible.

    # Convolutional layers
    # kernel_size: the size of the CNN filter of small matrix that has weights
    # kernel_size=3: the kernel/filter will have both height and width of 3 (i.e., 3×3).
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

    # Pooling layer
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # Fully-connected layers
    # # After two 2x2 poolings on a 28x28 input, the spatial dimension is reduced to 7x7.
    # Hence, the input to the first linear layer is 64 * 7 * 7.
    self.fc1 = nn.Linear(in_features=64 * 7 * 7, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

  def forward(self, x):
    # Convolution -> ReLU -> Pool
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))

    # Flatten
    x = x.view(x.size(0), -1)  # same as x.view(-1, 64*7*7)

    # Fully-connected -> ReLU
    x = F.relu(self.fc1(x))

    # Final layer (logits)
    x = self.fc2(x)

    return x


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SingleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # A single 2D convolution layer
        self.conv1 = nn.Conv2d(
            in_channels=4,
            # input channel size of one example is 1, i.e. grey scale
            # output channel size becomes 2.
            # out_channels=2, that convolution layer learns two distinct filters
            out_channels=4,
            kernel_size=3,
            padding=1
        )

    def forward(self, x):
        # Apply convolution (and optionally an activation)
        # Here we show it with a ReLU, but you can remove F.relu(...) if you want pure convolution
        #So if x has shape [batch_size, D1, D2, ..., Dn],
        # this operation will reshape x into [batch_size, D1,D2 * ... * Dn].
        # x = torch.flatten(x, start_dim=2, end_dim=-1)
        x=self.conv1(x)
        # x = F.relu(self.conv1(x))
        return x


In [4]:
# if __name__ == "__main__":
#     # Create a random batch of 1 image, 4×4 (grayscale)
#     x = torch.randn((1, 1, 4, 4))

#     model = SingleConvNet()
#     output = model(x)

#     print("Input shape:", x.shape)
#     print("Input tensor:\n", x)        # Show the actual input values

#     print("\nOutput shape:", output.shape)
#     print("Output tensor:\n", output)  # Show the actual output values

#     print("Weights:\n", model.conv1.weight)
#     print("\nBias:\n", model.conv1.bias)
#     print("\nPadding:\n", model.conv1.padding)

# # Weights, bias are random

In [5]:
cnn = SingleConvNet()
mg = MaseGraph(model=cnn)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 2
# x = torch.randn((batch_size, 2, 2))
# batch_size: number of examples
# 1, 4,4,: each example has 1 channel, each channel has a size of 4x4
x = torch.randn((batch_size, 4, 4, 4))
# print(x)
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)
# for node in mg.nodes:
#         mase_op = node.meta["mase"]["common"]["mase_op"]
#         print ("node.op:", node.op)
#         print ('mase_op:', mase_op)
#         print ("common:",node.meta["mase"]["common"])



DEBUG    graph():
    %x : [num_users=1] = placeholder[target=x]
    %conv1 : [num_users=1] = call_module[target=conv1](args = (%x,), kwargs = {})
    return conv1


Hellos in add_common_metadata
sigoyi in add_common_metadata
graph_model:  GraphModule(
  (conv1): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)



def forward(self, x):
    conv1 = self.conv1(x);  x = None
    return conv1
    
# To see more debug info, please use `graph_module.print_readable()`
wocao in add_common_metadata
graph_iterator_for_metadata:  GraphModule(
  (conv1): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)



def forward(self, x):
    conv1 = self.conv1(x);  x = None
    return conv1
    
# To see more debug info, please use `graph_module.print_readable()`
args in call_module:  (tensor([[[[ 0.2886,  0.3866, -0.2011, -0.1179],
          [-0.8294, -1.4073,  1.6268,  0.1723],
          [-0.7043,  0.3147,  0.1574,  0.3854],
          [ 0.5737,  0.9979,  0.5436,  0.0788]],

         [[ 0.9985, -0.4987,  0.7611,  0.6183],
          [-0.2994, -0.1878,  1.9159,  0.6902],
          [-0.3140, -1.0787,  0.2408, -1.3962],
          [ 0.1136, 

# Quantize

In [6]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
# Fixed.toml used to quantize the model
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"]["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 5]
    for result, result_info in node.meta["mase"]["common"]["results"].items():
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 5]

placeholder
node_config:  {'name': 'fixed', 'data_in_width': 8, 'data_in_frac_width': 5, 'weight_width': 8, 'weight_frac_width': 5, 'bias_width': 8, 'bias_frac_width': 5, 'data_out_width': 8, 'data_out_frac_width': 5, 'floor': True}


INFO     Inspecting graph [add_common_node_type_analysis_pass]
INFO     
Node name    Fx Node op    Mase type            Mase op      Value type
-----------  ------------  -------------------  -----------  ------------
x            placeholder   placeholder          placeholder  NA
conv1        call_module   module_related_func  conv2d       fixed
output       output        output               output       NA


output


# Hardware Metapass

In [7]:
mg, _ = add_hardware_metadata_analysis_pass(mg)
for node in mg.nodes:
        mase_op = node.meta["mase"]["common"]["mase_op"]
        print ("node.op:", node.op)
        print ('mase_op:', mase_op)
        print ("common:",node.meta["mase"]["common"])
        print ("hardware:",node.meta["mase"]["hardware"])

for node in mg.fx_graph.nodes:
        if node.meta["mase"].parameters["hardware"]["is_implicit"]:
            continue
        # Only modules have internal parameters
        if node.meta["mase"].module is None:
            continue
        # print (node.meta["mase"].parameters["hardware"])
        # Only checks the hardware data that contains the key toolchain
        if "INTERNAL" in node.meta["mase"].parameters["hardware"]["toolchain"]:
                for param_name, parameter in node.meta["mase"].module.named_parameters():
                        print ("param_name in CNN.jynb:",param_name)
                        print ("parameter in CNN.jynb:", parameter)

"""
weights and bias in the Conv/linear in Maze
param_data = node.meta["mase"].module.get_parameter(param_name).data
print ("param_data: ", param_data)
"""


mase_op: conv2d
2222
 I changed max_parallelism add_hardware_metadata_analysis_pass
vp:  {}
arg in add_verilog_param: data_in_0
vp after for:  {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 4, 'DATA_IN_0_PARALLELISM_DIM_1': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_2': 4, 'DATA_IN_0_PARALLELISM_DIM_2': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_3': 2, 'DATA_IN_0_PARALLELISM_DIM_3': 2}
arg in add_verilog_param: weight
vp after for:  {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 4, 'DATA_IN_0_PARALLELISM_DIM_1': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_2': 4, 'DATA_IN_0_PARALLELISM_DIM_2': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_3': 2, 'DATA_IN_0_PARALLELISM_DIM_3': 2, 'WEIGHT_PRECISION_0': 8, 'WEIGHT_PRECISION_1': 5, 'WEIGHT_TENSOR_SIZE_DIM_0': 3, 'WEIGHT_PARALLELISM_DIM_0': 2, 'WEIGHT_TENSOR_SIZE_DIM_1': 3, 'WEIGHT_

'\nweights and bias in the Conv/linear in Maze\nparam_data = node.meta["mase"].module.get_parameter(param_name).data\nprint ("param_data: ", param_data)\n'

# Emit for SV file

In [8]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

INFO     Emitting Verilog...
INFO     Emitting internal components...


Parameter_map in VerilogEmitter: {'conv1_DATA_IN_0_PRECISION_0': 8, 'conv1_DATA_IN_0_PRECISION_1': 5, 'conv1_DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'conv1_DATA_IN_0_PARALLELISM_DIM_0': 2, 'conv1_DATA_IN_0_TENSOR_SIZE_DIM_1': 4, 'conv1_DATA_IN_0_PARALLELISM_DIM_1': 2, 'conv1_DATA_IN_0_TENSOR_SIZE_DIM_2': 4, 'conv1_DATA_IN_0_PARALLELISM_DIM_2': 2, 'conv1_DATA_IN_0_TENSOR_SIZE_DIM_3': 2, 'conv1_DATA_IN_0_PARALLELISM_DIM_3': 2, 'conv1_WEIGHT_PRECISION_0': 8, 'conv1_WEIGHT_PRECISION_1': 5, 'conv1_WEIGHT_TENSOR_SIZE_DIM_0': 3, 'conv1_WEIGHT_PARALLELISM_DIM_0': 2, 'conv1_WEIGHT_TENSOR_SIZE_DIM_1': 3, 'conv1_WEIGHT_PARALLELISM_DIM_1': 2, 'conv1_WEIGHT_TENSOR_SIZE_DIM_2': 4, 'conv1_WEIGHT_PARALLELISM_DIM_2': 2, 'conv1_WEIGHT_TENSOR_SIZE_DIM_3': 4, 'conv1_WEIGHT_PARALLELISM_DIM_3': 2, 'conv1_BIAS_PRECISION_0': 8, 'conv1_BIAS_PRECISION_1': 5, 'conv1_BIAS_TENSOR_SIZE_DIM_0': 4, 'conv1_BIAS_PARALLELISM_DIM_0': 2, 'conv1_BIAS_TENSOR_SIZE_DIM_1': 1, 'conv1_BIAS_PARALLELISM_DIM_1': 1, 'conv1_STRIDE_TENSOR_S

# Memory

In [9]:
mg, _ = emit_bram_transform_pass(mg)

"""
  param_data:  tensor([[[[-0.1946,  0.2865,  0.1487],
          [ 0.1616,  0.0175, -0.1709],
          [ 0.0564, -0.3112, -0.2409]]],


        [[[-0.1718,  0.2103,  0.1954],
          [-0.1478, -0.0120,  0.2132],
          [ 0.3314,  0.1323,  0.0450]]]])
  This would be weight, as I defined conv as (2,1,3,3)
  The kernel filter size is 3x3, and we have 2 filters, thus 9x2=18 elements
  Bias: depends on number of filter-> here 2 filters, thus 2 bias elements,
    as bias added after filter multiplies with section of pixel
"""

INFO     Emitting BRAM...
DEBUG    Emitting DAT file for node: conv1, parameter: weight
DEBUG    ROM module weight successfully written into /root/.mase/top/hardware/rtl/conv1_weight_source.sv
DEBUG    Init data weight successfully written into /root/.mase/top/hardware/rtl/conv1_weight_rom.dat
DEBUG    Emitting DAT file for node: conv1, parameter: bias
DEBUG    ROM module bias successfully written into /root/.mase/top/hardware/rtl/conv1_bias_source.sv
DEBUG    Init data bias successfully written into /root/.mase/top/hardware/rtl/conv1_bias_rom.dat


1 in emit_bram_transform_pass
/root/.mase/top/hardware/rtl
param_name in emit_bram_handshake: weight
out_size: 16
out_size in emit_parameters_in_mem_internal: 16
conv1
verilog_param_name weight
total_size: 144
shape: [4, 4, 3, 3]
 node.meta[mase].parameters[hardware][verilog_param] {'DATA_IN_0_PRECISION_0': 8, 'DATA_IN_0_PRECISION_1': 5, 'DATA_IN_0_TENSOR_SIZE_DIM_0': 4, 'DATA_IN_0_PARALLELISM_DIM_0': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_1': 4, 'DATA_IN_0_PARALLELISM_DIM_1': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_2': 4, 'DATA_IN_0_PARALLELISM_DIM_2': 2, 'DATA_IN_0_TENSOR_SIZE_DIM_3': 2, 'DATA_IN_0_PARALLELISM_DIM_3': 2, 'WEIGHT_PRECISION_0': 8, 'WEIGHT_PRECISION_1': 5, 'WEIGHT_TENSOR_SIZE_DIM_0': 3, 'WEIGHT_PARALLELISM_DIM_0': 2, 'WEIGHT_TENSOR_SIZE_DIM_1': 3, 'WEIGHT_PARALLELISM_DIM_1': 2, 'WEIGHT_TENSOR_SIZE_DIM_2': 4, 'WEIGHT_PARALLELISM_DIM_2': 2, 'WEIGHT_TENSOR_SIZE_DIM_3': 4, 'WEIGHT_PARALLELISM_DIM_3': 2, 'BIAS_PRECISION_0': 8, 'BIAS_PRECISION_1': 5, 'BIAS_TENSOR_SIZE_DIM_0': 4, 'BIAS_PARALLELI

'\n  param_data:  tensor([[[[-0.1946,  0.2865,  0.1487],\n          [ 0.1616,  0.0175, -0.1709],\n          [ 0.0564, -0.3112, -0.2409]]],\n\n\n        [[[-0.1718,  0.2103,  0.1954],\n          [-0.1478, -0.0120,  0.2132],\n          [ 0.3314,  0.1323,  0.0450]]]])\n  This would be weight, as I defined conv as (2,1,3,3)\n  The kernel filter size is 3x3, and we have 2 filters, thus 9x2=18 elements\n  Bias: depends on number of filter-> here 2 filters, thus 2 bias elements,\n    as bias added after filter multiplies with section of pixel\n'

In [10]:
mg, _ = emit_cocotb_transform_pass(mg)

INFO     Emitting testbench...


In [12]:
from chop.actions import simulate

simulate(skip_build=False, skip_test=False)

INFO: Running command perl /usr/local/bin/verilator -cc --exe -Mdir /workspace/docs/labs/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/usr/local/lib/python3.11/dist-packages/cocotb/libs -L/usr/local/lib/python3.11/dist-packages/cocotb/libs -lcocotbvpi_verilator' -Wno-fatal -Wno-lint -Wno-style --trace-fst --trace-structs --trace-depth 3 -I/root/.mase/top/hardware/rtl -I/workspace/src/mase_components/interface/rtl -I/workspace/src/mase_components/language_models/rtl -I/workspace/src/mase_components/memory/rtl -I/workspace/src/mase_components/vivado/rtl -I/workspace/src/mase_components/convolution_layers/rtl -I/workspace/src/mase_components/cast/rtl -I/workspace/src/mase_components/systolic_arrays/rtl -I/workspace/src/mase_components/scalar_operators/rtl -I/workspace/src/mase_components/transformer_layers/rtl -I/workspace/src/mase_components/common/rtl -I/workspace/src/mase_components/hls/rtl -I/workspace/src/mase_components/v

INFO     Build finished. Time taken: 2.62s


make: Leaving directory '/workspace/docs/labs/sim_build'
sys in simulate.py:  <module 'sys' (built-in)>
cmd: [['/workspace/docs/labs/sim_build/top']]
INFO: Running command /workspace/docs/labs/sim_build/top in directory /workspace/docs/labs/sim_build
     -.--ns INFO     gpi                                ..mbed/gpi_embed.cpp:76   in set_program_name_in_venv        Did not detect Python virtual environment. Using system-wide Python interpreter
     -.--ns INFO     gpi                                ../gpi/GpiCommon.cpp:101  in gpi_print_registered_impl       VPI registered
     0.00ns INFO     cocotb                             Running on Verilator version 5.020 2024-01-01
     0.00ns INFO     cocotb                             Running tests with cocotb v1.8.0 from /usr/local/lib/python3.11/dist-packages/cocotb
     0.00ns INFO     cocotb                             Seeding Python random module with 1741614936
     0.00ns INFO     cocotb.regression                  Found test mase_top_

/workspace/src/mase_cocotb/driver.py:25: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._send_thread())
/workspace/src/mase_cocotb/monitor.py:27: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._recv_thread())
[W310 13:55:48.051530978 NNPACK.cpp:62] Could not initialize NNPACK! Reason: Unsupported hardware.


2000020.00ns INFO     cocotb.regression                  test failed
                                                         Traceback (most recent call last):
                                                           File "/root/.mase/top/hardware/test/mase_top_tb/test.py", line 28, in test
                                                             await tb.wait_end(timeout=2, timeout_unit="ms")
                                                           File "/workspace/src/mase_cocotb/testbench.py", line 76, in wait_end
                                                             raise TimeoutError("Timed out waiting for test to end.")
                                                         TimeoutError: Timed out waiting for test to end.
2000020.00ns INFO     cocotb.regression                  **************************************************************************************
                                                         ** TEST                          STATUS  SI

/usr/lib/python3.11/tempfile.py:1073: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpf8ni7col'>
  _warnings.warn(warn_message, ResourceWarning)


- :0: Verilog $finish


INFO     Test finished. Time taken: 48.98s


INFO: Results file: /workspace/docs/labs/sim_build/results.xml
